# Importing all relevant libraries

In [670]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
from bs4 import BeautifulSoup
import urllib3
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Using BeautifulSoup to scrape the webpage for the relevant data, in this case the table of postcodes


In [ ]:

http = urllib3.PoolManager()

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = http.request('GET', url)
soup = BeautifulSoup(response.data)



In [672]:
#Extracting Column Names
table_head = soup.find('tr')
header=table_head.text.split('\n')
header=header[1:4]





# Data extraction and processing

In [673]:
#Extracting Data
data=soup.find('tbody').text.strip().split('\n')

dataf = [data[x:x+5] for x in range(0, len(data),5)]
df=pd.DataFrame(dataf)
df.head()
df=df.drop(df.index[0])
df=df.drop([3,4], axis=1)
df.columns=header
df.head()
df = df[df['Borough']!='Not assigned']
dfn=df

In [674]:
#Data Processing, removing rows where boroughs not assigned 
df = df[df['Borough']!='Not assigned']
df.loc[df['Neighbourhood']=='Not assigned', 'Neighbourhood'] = df[df['Neighbourhood']=='Not assigned']['Borough']

for postcode,g in df.groupby('Postcode'):
    df.loc[df['Postcode']==postcode, 'Neighbourhoods'] = ','.join(g['Neighbourhood'].tolist())
   

 
df=df[['Postcode','Borough','Neighbourhoods']]



df.drop_duplicates('Postcode',keep='first',inplace=True)
df.reset_index(drop=True)

,Postcode,Borough,Neighbourhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [675]:
df.shape

(103, 3)

In [676]:
#Sorting loaded data by postal code

geodf=pd.read_csv('Geospatial_Coordinates.csv')
geodf.shape
geodf=geodf.sort_values(by='Postal Code')


#sorting scraped data by postal code and combining
tst=df.sort_values(by='Postcode')
tst.loc(axis=1)['Latitude']=geodf[['Latitude']].values
tst.loc(axis=1)['Longitude']=geodf[['Longitude']].values

,Postcode,Borough,Neighbourhoods
12,M1B,Scarborough,"Rouge,Malvern"
28,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
43,M1E,Scarborough,"Guildwood,Morningside,West Hill"
54,M1G,Scarborough,Woburn
63,M1H,Scarborough,Cedarbrae
77,M1J,Scarborough,Scarborough Village
92,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
108,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
124,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
141,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [712]:
#to be used later in clustering
dft=tst

In [717]:
#final dataframe
tst=tst.reset_index(drop=True)
tst

,Postcode,Borough,Neighbourhoods,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Visualizing the postal code layout in Toronto

In [727]:
import folium
# creating map of Toronto using latitude and longitude values
latitude=43.6532
longitude=-79.3832
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# plotting coordinates of postal code, with Neighbourhood, Borough as labels in popup
for lat, lng, borough, neighborhood in zip(tst['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighbourhoods']):
    label = 'Neighbourhoods contained:{}Borough: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto

#sorted=dft[dft.Borough.str.contains('Toronto')]

# Using Foursquare API to get venues around the Post Code

In [ ]:


#Pulling venues in a kilometer radius of postal code center
CLIENT_ID = 'QIJAH1HDYSODDWAH4UOHSNJGGNXD1D5WOLU3SFOUWCRRI53T' 
CLIENT_SECRET = '441FOL4XTMQSK4F3YPBRQVR2YN4ESJQL40ALSBCBQMM0U20T' 
VERSION = '20180605'
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)




        
            
           
            
    





In [708]:
#Applying getNearbyVenues to postcodes
toronto_venues = getNearbyVenues(names=dft['Postcode'],
                                   latitudes=dft['Latitude'],
                                   longitudes=dft['Longitude']
                                  )

In [686]:
#Kowalski.. Analysis
toronto_venues.groupby('Postcode').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,7,7,7,7,7,7
M1G,3,3,3,3,3,3
M1H,7,7,7,7,7,7
M1J,2,2,2,2,2,2
M1K,6,6,6,6,6,6
M1L,10,10,10,10,10,10
M1M,3,3,3,3,3,3


In [687]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


In [728]:
#One Hot Encoding for venue categories to be used later
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

In [729]:
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0

In [730]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()


In [691]:
#Top five venues per Postcode and frequency
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metro Station   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----M1C----
                 venue  freq
0       History Museum   0.5
1                  Bar   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----M1E----
                 venue  freq
0          Pizza Place  0.14
1  Rental Car Location  0.14
2       Breakfast Spot  0.14
3       Medical Center  0.14
4    Electronics Store  0.14


----M1G----
                 venue  freq
0          Coffee Shop  0.67
1    Korean Restaurant  0.33
2    Accessories Store  0.00
3                Motel  0.00
4  Monument / Landmark  0.00


----M1H----
                  venue  freq
0  Caribbean Restaurant  0.14
1                Bakery  0.14
2       Thai Restaurant  0.14
3    Athletics & Sports  0.14
4   Fried Chic

            venue  freq
0     Coffee Shop  0.13
1           Hotel  0.05
2        Aquarium  0.05
3            Café  0.04
4  Scenic Lookout  0.03


----M5K----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.08
2               Hotel  0.06
3          Restaurant  0.04
4  Italian Restaurant  0.03


----M5L----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.06
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----M5M----
                  venue  freq
0  Fast Food Restaurant  0.09
1    Italian Restaurant  0.09
2           Coffee Shop  0.09
3     Indian Restaurant  0.04
4               Butcher  0.04


----M5N----
                 venue  freq
0               Garden   0.5
1                 Pool   0.5
2   Mexican Restaurant   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----M5P----
               venue  freq
0               Park  0.25
1   Sushi Restaurant  0.25
2      Je

In [692]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [731]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)



In [694]:
neighborhoods_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M1C,History Museum,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,M1E,Medical Center,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Rental Car Location,Mexican Restaurant,Dog Run,Dim Sum Restaurant,Diner
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store
5,M1J,Playground,Smoke Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
6,M1K,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,M1L,Bakery,Bus Line,Park,Fast Food Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Yoga Studio,Doner Restaurant
8,M1M,Motel,Movie Theater,American Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
9,M1N,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [695]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 4, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [697]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



,Cluster Labels,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M1B,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,4,M1C,History Museum,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
2,1,M1E,Medical Center,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Rental Car Location,Mexican Restaurant,Dog Run,Dim Sum Restaurant,Diner
3,1,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,1,M1H,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store
5,1,M1J,Playground,Smoke Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
6,1,M1K,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,1,M1L,Bakery,Bus Line,Park,Fast Food Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Yoga Studio,Doner Restaurant
8,1,M1M,Motel,Movie Theater,American Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
9,1,M1N,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [698]:
toronto_merged = dft
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')
toronto_merged.head() 

,Postcode,Borough,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
28,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,History Museum,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
43,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Medical Center,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Rental Car Location,Mexican Restaurant,Dog Run,Dim Sum Restaurant,Diner
54,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
63,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store


In [699]:
toronto_merged[['Cluster Labels']]=toronto_merged[['Cluster Labels']].fillna(0,downcast='infer')
#np.nan_to_num(toronto_merged['Cluster Labels']).astype(int)
toronto_merged

,Postcode,Borough,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
28,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4,History Museum,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
43,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1,Medical Center,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Rental Car Location,Mexican Restaurant,Dog Run,Dim Sum Restaurant,Diner
54,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
63,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store
77,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Playground,Smoke Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
92,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,1,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
108,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Park,Fast Food Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Yoga Studio,Doner Restaurant
124,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,1,Motel,Movie Theater,American Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
141,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,1,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [700]:
#Creating map with clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [702]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
206,Scarborough,0,Park,Coffee Shop,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
247,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,North York,0,Convenience Store,Park,Bank,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
3,North York,0,Food & Drink Shop,Bus Stop,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
96,North York,0,Park,Airport,Snack Place,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
113,North York,0,Convenience Store,Shopping Mall,Park,Bank,Hotel,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
83,East York,0,Park,Coffee Shop,Convenience Store,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
145,Central Toronto,0,Park,Lawyer,Swim School,Bus Line,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
240,Downtown Toronto,0,Park,Trail,Playground,Building,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
158,Central Toronto,0,Park,Trail,Sushi Restaurant,Jewelry Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [701]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Scarborough,1,Medical Center,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Rental Car Location,Mexican Restaurant,Dog Run,Dim Sum Restaurant,Diner
54,Scarborough,1,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
63,Scarborough,1,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store
77,Scarborough,1,Playground,Smoke Shop,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
92,Scarborough,1,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
108,Scarborough,1,Bakery,Bus Line,Park,Fast Food Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Yoga Studio,Doner Restaurant
124,Scarborough,1,Motel,Movie Theater,American Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
141,Scarborough,1,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
152,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Furniture / Home Store,Chinese Restaurant,Pet Store,Comfort Food Restaurant,Comic Shop,Ethiopian Restaurant,Empanada Restaurant
165,Scarborough,1,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Vietnamese Restaurant,Bakery,Shopping Mall,Sandwich Place,Yoga Studio,Doner Restaurant,Diner


In [703]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,2,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [704]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
139,North York,3,Baseball Field,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop


In [705]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Scarborough,4,History Museum,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
